In [92]:
# Using simplegmail abstraction instead of writing gmail logic from scratch
# https://github.com/jeremyephron/simplegmail
from simplegmail import Gmail, query
# ollama
import ollama


In [72]:
# Get gmail service - may need to authenticate - check https://github.com/jeremyephron/simplegmail
gmail = Gmail()

In [40]:
# Get mails from past _ units
numberOf = 1
messages = gmail.get_messages(query=query.construct_query(newer_than=(numberOf, "hour"), labels=[["INBOX"]]))

In [105]:
print(f'Total messages: {len(messages)}')
for message in messages:
    print(f"""from: {message.sender} date:{message.date}
        subject: {message.subject}""")
    print(message.plain)

Total messages: 8
from: <Recruiting@tranetechnologies.com> date:2024-08-15 19:43:11-04:00
        subject: Trane Technologies Candidate Experience Survey
None
from: Platform Notifications <PlatformNotifications-noreply@google.com> date:2024-08-15 18:04:00-04:00
        subject: Your project gmail-api-432621 has been shut down
Project Permanent Deletion Warning


Dear Developer,

Your Google Cloud project gmail-api-432621 was shut down on August 15, 2024  
10:03:58 PM UTC.

Shut-down projects and the data stored in them may be recovered for a  
limited time before the projects are permanently deleted. If you'd like to  
recover your project, you must cancel the project's permanent deletion  
before September 14, 2024 10:03:58 PM UTC.

To recover your shut-down project:

Visit the Resources pending deletion page.
Select the project you want to recover, and click Restore.
In the confirmation dialog, click Restore.


If you cannot recover your project by following the instructions above, y

In [103]:
print(f'Total messages: {len(messages)}')
for message in messages:
    print(f"""from: {message.sender} date:{message.date}
        subject: {message.subject}""")
    result = ollama.generate(model='llama3.1', prompt=f"""Here is mail I received:
{message.plain}
Give a simple Yes or No answer to the question - Do you think the mail I recieved is regarding a software job application? """)
    print(result['response'])
    

Total messages: 8
from: <Recruiting@tranetechnologies.com> date:2024-08-15 19:43:11-04:00
        subject: Trane Technologies Candidate Experience Survey
No.
from: Platform Notifications <PlatformNotifications-noreply@google.com> date:2024-08-15 18:04:00-04:00
        subject: Your project gmail-api-432621 has been shut down
No.
from: Sydney Christensen <Sydney.Christensen@mondo.com> date:2024-08-15 17:45:59-04:00
        subject: Digital Content Producer - Remote - Contract Opportunity!!
No.
from: Bhanu Singh <bhanupriya@ibridgetechsoft.com> date:2024-08-15 15:29:47-04:00
        subject: Python Developer with NodeJs
No.
from: "People, Culture and Belonging: People Experience" <human.resources@message.vu.edu> date:2024-08-15 13:41:04-04:00
        subject: Talent Acquisition Satisfaction Survey
No.
from: Bhanu Singh <bhanupriya@ibridgetechsoft.com> date:2024-08-15 11:04:08-04:00
        subject: Java developer (Healthcare domain)
No.
from: Trane Technologies Careers <tranetechnologies